In [ ]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

In [ ]:
# importing dataset
def loadDataset():
    trainDataset = h5py.File("datasets/train_catvnoncat.h5", "r")
    trainDatasetX = np.array(trainDataset["train_set_x"][:])
    trainDatasetY = np.array(trainDataset["train_set_y"][:])
    
    testDataset = h5py.File("datasets/test_catvnoncat.h5")
    testDatasetX = np.array(testDataset["test_set_x"][:])
    testDatasetY = np.array(testDataset["test_set_y"][:])

    classes = np.array(trainDataset["list_classes"][:])

    trainDatasetY = trainDatasetY.reshape((1, trainDatasetY.shape[0]))
    testDatasetY = testDatasetY.reshape((1, testDatasetY.shape[0]))
    return trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes

trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes = loadDataset()
print(trainDatasetX.shape, trainDatasetY.shape)

In [ ]:
index = 27
plt.imshow(trainDatasetX[index])
plt.show()

In [ ]:
#getting parameters
m_train = trainDatasetX.shape[0]
num_px = trainDatasetX.shape[1]
print("Number of training examples: ", m_train)
print("Height of image: ", num_px, "px")
print("Shape of image: ", trainDatasetX[0].shape)
print("Shape of training dataset X: ", trainDatasetX.shape)
print("Shape of training dataset Y: ", trainDatasetY.shape)

In [ ]:
#reshaping and flattening dataset
XTrainFlatten = trainDatasetX.reshape((trainDatasetX.shape[0], -1)).T
XTestFlatten = testDatasetX.reshape((testDatasetX.shape[0], -1)).T
print("Shape of XFlatten: ", XTrainFlatten.shape)
print("Shape of XTestFlatten: ", XTestFlatten.shape)

In [ ]:
#standardizing the images
XTrainFlatten = XTrainFlatten / 255
XTestFlatten = XTestFlatten / 255

In [ ]:
#Sigmoid Function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
#initializing parameters
def initializeParameters(dim):
    w = np.zeros((dim, 1))
    b = 0
    return w, b

In [ ]:
#forward and backward propagation
def propagate(w, b, X, Y):
    m = X.shape[1]

    #forward prop
    Z = w.T.dot(X) + b
    A = sigmoid(Z)
    cost = -1/m * np.sum( Y * np.log(A) + (1-Y) * np.log(1-A))

    #backward prop
    dz = A - Y
    db = 1/m * np.sum(dz)
    dw = 1/m * X.dot(dz.T)
    grad = {"dw": dw,
            "db": db}
    
    return grad, cost

In [ ]:
#gradient descent 
def gradientDescent(X, Y, numIterations, learningRate, printCost = False):
    nx = X.shape[0]
    m = X.shape[1]
    w, b = initializeParameters(nx)
    costs = []

    for i in range(numIterations):
        grad, cost = propagate(w, b, X, Y)
        costs.append(cost)
        w -= grad["dw"] * learningRate
        b -= grad["db"] * learningRate

        if(printCost and i%100==0):
            print(f"Cost after {i} iterations: {cost}")
    
    params = {"w": w,
              "b": b}
    
    return params, costs

In [ ]:
#running gradient descent on our dataset
params, costs = gradientDescent(XTrainFlatten, trainDatasetY, numIterations = 2000, learningRate = 0.005, printCost = True)

In [ ]:
#plotting costs
plt.plot(costs)
plt.show()

In [ ]:
#predictions
def predict(params, X, Y):
    w = params["w"]
    b = params["b"]
    Z = w.T.dot(X) + b
    A = sigmoid(Z)
    Y_Prediction = (A > 0.5) * 1.0
    return Y_Prediction

#checking accuracy
trainPrediction = predict(params, XTrainFlatten, trainDatasetY)
testPrediction = predict(params, XTestFlatten, testDatasetY)
print(f"train accuracy: {100-np.mean(np.abs(trainPrediction-trainDatasetY))*100}")
print(f"test accuracy: {100-np.mean(np.abs(testPrediction-testDatasetY))*100}")

Number of training examples:  209 \
Height of image:  64 px \
Shape of image:  (64, 64, 3) \
Shape of training dataset X:  (209, 64, 64, 3) \
Shape of training dataset Y:  (1, 209) \
Shape of XFlatten:  (12288, 209) \
Shape of XTestFlatten:  (12288, 50) 
### train accuracy: 99.04306220095694
### test accuracy: 70.0